In [1]:
import pandas as pd
import sys
from datetime import datetime
import numpy as np
from DataScience.Utils import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

version = 'V3_ConProtocolo'

path_minado = "DataVisualization/Metricas_minadas/trace_activities.csv"
path_orig = "Salida_script/RiberaSalud_"+str(version)+"_spanish.csv"

In [2]:
ribera = pd.read_csv(path_minado, sep = ',')
ribera_dicc = {'Ribera V3':ribera}

In [3]:
#Información dataframes
for df_name, df in zip(ribera_dicc.keys() , ribera_dicc.values()):
    df_info(df_name, df, atrib_name = 'CodPaciente', only_basics= True)


- Ribera V3:
    Ribera V3 -> (2375, 49)
    WARNING! -> El df Ribera V3 no tiene una columna llamada CodPaciente


In [4]:
ribera.columns

Index(['traceId', 'index', 'activity', 'start', 'end', 'duration',
       'ADAdministrado', 'ADMotivoNoAdmon', 'CodEpisodio', 'CodIntervencion',
       'CodPrimerEpisodio', 'DuracionAdmMedCirugia', 'HabitacionQuirofano',
       'INGCopiaTratamientoIngreso', 'INGDesTipoIngreso', 'MEDCadaXHoras',
       'MEDCondicion', 'MEDFecIniAdmon', 'MEDNumTomasDia', 'MedicalServiceId',
       'POSCadaXHoras', 'POSCondicion', 'POSDescMedicamento', 'POSDiaFinal',
       'POSDiaInicial', 'POSDiasSemana', 'POSDosis', 'POSDosisDetalle',
       'POSDuracion', 'POSEliminado', 'POSFrecuenciaDosis',
       'POSInstruccionesPaciente', 'POSNumTomasDia', 'POSUrgente',
       'PREOPCirujano', 'PREOPCodIntervencion', 'PREOPCodProcedimiento',
       'PREOPDescDiagnostico', 'PREOPDescServDestino', 'PREOPDescripcion',
       'PREOPTAnestesia', 'PREOPTipoEpisodioIntervencion',
       'PREOPTipoEpisodioOrigen', 'PROTCuidados', 'Perdidaprotocolo1',
       'Perdidaprotocolo2', 'Reservaquirofano', 'VariantId',
       'Va

In [5]:
def conteo_atributos_Traza(df, atribb):
    df_grouped = df.groupby(atribb)['CodPaciente'].nunique()
    print(df_grouped.apply(lambda x : f'{x} trazas ({((x/df.CodPaciente.nunique())*100):.2f}%)'))
    print('........................................................................................................................................................\n')
    
lista_atributos_categoricos = ['Actividad', 'INGDesTipoIngreso', 'INGCopiaTratamientoIngreso',
            'ADAdministrado','ADMotivoNoAdmon', 'MEDCadaXHoras', 'MEDNumTomasDia', 'MEDCondicion',
            'PREOPTipoEpisodioIntervencion','PREOPDescripcion', 'PREOPDescDiagnostico', 'PREOPTAnestesia','PREOPDescServDestino',
            'PREOPCirujano', 'Reserva_quirofano', 'Varios_quirofano', 'Habitacion Quirofano', 'PROTCuidados','POSDescMedicamento',
            'POSDosisDetalle','POSInstruccionesPaciente',
            'Perdida_protocolo 1', 'Perdida_protocolo 2']


# Guardar la salida original de stdout para poder restaurarla más tarde
stdout_original = sys.stdout
# Abrir un archivo en modo de escritura. Si el archivo no existe, se creará.
with open('Conteo_atributos_porTraza.txt', 'w') as f:
    # Redirigir stdout al archivo
    sys.stdout = f
    # Llamar a la función cuya salida queremos capturar
    for atrib in lista_atributos_categoricos:
        conteo_atributos_Traza(ribera, atrib)
    # Opcional: cualquier otra cosa que imprimas aquí también se escribirá en el archivo
# Restaurar stdout a su estado original
sys.stdout = stdout_original
print("Esta salida se mostrará en la terminal.")

##### EDA - RIBERA SALUD V3
Preguntas importantes a hacerse en este dataset
 - Pregunta 1: 
 - Pregunta 2:

In [5]:
# Convertir la columna booleana a numérica
ribera['Perdida_protocolo 1'] = ribera['Perdida_protocolo 1'].astype('Int64')
#Obtenemos los atributos numericos
columnas_categoricas = ribera.select_dtypes(include=['object', 'bool']).columns
columnas_numericas = ribera.select_dtypes(include=['int64', 'float64']).columns
ribera_categoricas = ribera[columnas_categoricas]
ribera_numericas = ribera[columnas_numericas]

ribera_categoricas.drop(['Perdida_protocolo 2', 'CodEpisodio' ,'CodPrimerEpisodio' , 'FECHA',  'FECHA_FIN', 'MEDFecIniAdmon' ], axis = 1, inplace=True)
ribera_numericas.info()

KeyError: 'Perdida_protocolo 1'

In [6]:

#ribera = pd.get_dummies(ribera)
#preprocesado
# Reemplazar infinitos por un valor numérico grande o por NaN para luego llenarlo como arriba
ribera_numericas.replace([np.inf, -np.inf], np.nan, inplace=True)
# Para columnas numéricas, puedes llenar los NaN con la media o la mediana
for columna in ribera_numericas.columns:
    
    ribera_numericas[columna].fillna(ribera_numericas[columna].median(), inplace=True)
    if ribera_numericas[columna].dtype == 'float64':
        ribera_numericas[columna].astype('float16')
X = ribera_numericas.dropna(axis = 1, how = 'all')
#Escalar datos
scaler = StandardScaler()
ribera_escalado = scaler.fit_transform(ribera.select_dtypes(include=[np.number]))
# Verificar si aún existen NaN o infinitos en X
print("NaN en X:", ribera_numericas.isnull().any().any())
print("\nfloat64 en X:", ribera_numericas.info())
print("\nInfinitos en X:", np.isinf(ribera_numericas.select_dtypes(include=[np.number])).any().any())

NameError: name 'ribera_numericas' is not defined

In [7]:
#Realizamos una correlacion a todo el dataset (solo columnas numericas)
#Elimino valores nulos
print(ribera_numericas.isnull().values.any())
ribera_numericas_limpio = ribera_numericas.fillna(0)
print(ribera_numericas_limpio.isnull().values.any())
corre = ribera_numericas_limpio.corr()
#print(corre)
plt.figure(figsize=(30, 15))
sns.heatmap(corre, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.show()

NameError: name 'ribera_numericas' is not defined

In [ ]:
#Realizamos una correlacion a dataset con perdida de protocolo (solo columnas numericas)
#Elimino valores nulos
print(ribera_numericas.isnull().values.any())
ribera_numericas_limpio = ribera_numericas#.fillna(1)
print(ribera_numericas_limpio.isnull().values.any())
corre = ribera_numericas_limpio.corr()
#print(corre)
plt.figure(figsize=(30, 15))
sns.heatmap(corre, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.show()

In [ ]:
print(ribera.ADAdministrado.unique())
#med_sin1 = medicamentos.query('ADAdministrado != 0').count()
# ADAdministrado, ADMotivoNoAdmon, MEDCadaXHoras, MEDNumTomasDia, MEDCondicion
print(ribera.groupby('MEDNumTomasDia')['MEDCadaXHoras'].describe(), '\n----------------------------------------------------------------------------------')
print(ribera.groupby('MEDNumTomasDia')['MEDCondicion'].describe(), '\n----------------------------------------------------------------------------------')
print(ribera.groupby('MEDNumTomasDia')['CodPaciente'].describe(), '\n----------------------------------------------------------------------------------')

### Analisis perdida de protocolo

In [ ]:
resultante_1 = ribera.groupby('Perdida_protocolo 1')['CodPaciente'].nunique()
print('Perdida protocolo solo tiempo:\n',resultante_1)
print(f'Porcentaje de Perdidas de protocolo {(resultante_1[1] / resultante_1[0])*100:.2f}%')

resultante_2 = ribera.groupby('Perdida_protocolo 2')['CodPaciente'].nunique()
print('\nPerdida protocolo tiempo y otros condicionantes:\n',resultante_2)
print(f'Porcentaje de Perdidas de protocolo  {(resultante_2[1] / resultante_2[0])*100:.2f}%')

In [ ]:
ribera = pd.read_csv(path_minado, sep = ',')
ribera_dicc = {'Ribera V3':ribera}
ribera.columns

In [ ]:
#Obtenemos los atributos numericos
columnas_categoricas = ribera.select_dtypes(include=['object', 'bool']).columns
columnas_numericas = ribera.select_dtypes(include=['int64', 'float64']).columns
ribera_categoricas = ribera[columnas_categoricas]
ribera_numericas = ribera[columnas_numericas]

#ribera_categoricas = ribera_categoricas.drop(['PREOPCodProcedimiento','PREOPTAnestesia', 'traceId', 'Perdidaprotocolo2', 'PROTCuidados', 'CodEpisodio' ,'CodPrimerEpisodio' , 'start',  'end', 'MEDFecIniAdmon', 'VariantId', 'Variosquirofano', 'Reservaquirofano'], axis = 1)
ribera_numericas = ribera_numericas.drop(['DuracionAdmMedCirugia', 'index'], axis = 1)
ribera_fix = ribera.drop(['PREOPCodIntervencion', 'duration', 'PREOPCodProcedimiento', 'PREOPTAnestesia','CodIntervencion', 'Perdidaprotocolo2','PROTCuidados','CodEpisodio' ,'CodPrimerEpisodio' , 'start',  'end', 'MEDFecIniAdmon', 'VariantId', 'Variosquirofano', 'DuracionAdmMedCirugia', 'index', 'Reservaquirofano'], axis = 1)
ribera_fix.columns

In [ ]:
#df_perdidaprotocolo1 = ribera.loc[ribera['Perdida_protocolo 1'] == True]
import math
def perdida_protocolo_relaciones(df, atrib, detalle = False):

    if detalle == False:
        if atrib != 'traceId' and atrib != 'Perdidaprotocolo1':
            print(f"Trazas unicas cada columna:\n{df[atrib].nunique()}")
            #valores atributo con perdida de protocolo
            atributo_perdida = df.loc[df['Perdidaprotocolo1'] == 1]
            #print(f"Trazas unicas con perdida de protocolo{atributo_perdida.groupby(atrib)['traceId'].nunique()}\n") #({(atributo_perdida.groupby(atrib)['traceId'].nunique()/df['traceId'].nunique())*100:.2f}%)\n")
    
    if detalle:
        try:
            if df[atrib].isna().any():
                #print('tiene nulos')
                valores_atributo = df[atrib].unique().tolist()
                valores_atributo = valores_atributo[1:]
            else:
                valores_atributo = df[atrib].unique().tolist()
            #print(valores_atributo)
            for values in valores_atributo:
                #valores atributo normal
                atributo = df.loc[df[atrib] == values]
                print(f"El valor #{values}# de la columna #{atrib}# tiene {atributo['traceId'].nunique()} trazas unicas ({(atributo['traceId'].nunique()/df['traceId'].nunique())*100:.2f}%)")
                #valores atributo con perdida de protocolo
                atributo_perdida = atributo.loc[atributo['Perdidaprotocolo1'] == 1]
                print(f"El valor {values} de la columna {atrib} tiene {atributo_perdida['traceId'].nunique()} trazas unicas con perdida de protocolo ({(atributo_perdida['traceId'].nunique()/atributo['traceId'].nunique())*100:.2f}%)\n")
        except KeyError:
            print(atrib)
        except ZeroDivisionError:
            pass

columnas = ribera_fix.columns.to_list()
#ribera_medicamento = ribera_fix.groupby('traceId')['activity'] == 'Administración de Medicamentos'

for columna in columnas:
    print(columna,': \n')
    perdida_protocolo_relaciones(ribera_fix, columna, detalle= True)

    #print(ribera_fix[columna].value_counts())

In [ ]:
lista = [None, 1.0, 0.0]
lista